## 1. 필수 라이브러리 및 모듈 임포트

In [11]:
# Microsoft Agent Framework의 GroupChat 관련 모듈들
import asyncio
import logging
import os
from typing import cast
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

from agent_framework import (
    AgentRunUpdateEvent,
    ChatAgent,
    ChatMessage,
    GroupChatBuilder,
    Role,
    WorkflowOutputEvent,
)

# Azure OpenAI 클라이언트 사용
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


## 2. Azure OpenAI 설정

GroupChat 워크플로우에서 사용할 Azure OpenAI 클라이언트를 설정합니다.
- `AzureOpenAIChatClient`: Azure OpenAI 서비스를 통한 채팅 및 조율 작업용
- 환경변수 또는 Azure CLI 인증을 통한 연결 설정

In [12]:
# 환경 변수 확인
print("환경 변수 확인:")
env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY", 
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

for var in env_vars:
    value = os.getenv(var)
    if value:
        # API 키는 부분적으로만 표시
        if "API_KEY" in var:
            print(f"✅ {var}: {value[:8]}...")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: 설정되지 않음")

print("\n💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.")

환경 변수 확인:
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...
✅ AZURE_OPENAI_API_VERSION: 2024-08-01-preview
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1

💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.


### Azure OpenAI 설정 예제

Azure OpenAI를 사용하기 위해 다음 환경변수들을 `.env` 파일에 설정하세요:

```bash
# .env 파일 예제
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key-here
AZURE_OPENAI_API_VERSION=2024-06-01
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4o
```

또는 Azure CLI 인증을 사용할 수 있습니다:
```bash
az login
```

## 3. Azure OpenAI 클라이언트 생성 함수

환경변수 또는 Azure CLI 인증을 사용하여 클라이언트를 생성하는 헬퍼 함수를 정의합니다.

In [13]:
def _get_chat_client() -> AzureOpenAIChatClient:
    """
    Azure OpenAI 클라이언트 생성 헬퍼 함수
    
    환경변수가 설정된 경우 API 키 인증을 사용하고,
    그렇지 않으면 Azure CLI 자격 증명을 사용합니다.
    """
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    
    if api_key and endpoint and deployment_name:
        print("🔑 API 키 인증을 사용합니다.")
        return AzureOpenAIChatClient(
            endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
            deployment_name=deployment_name
        )
    else:
        print("🔐 Azure CLI 인증을 사용합니다.")
        return AzureOpenAIChatClient(
            credential=AzureCliCredential()
        )

# 클라이언트 테스트
try:
    test_client = _get_chat_client()
    print("✅ Azure OpenAI 클라이언트 생성 성공!")
except Exception as e:
    print(f"❌ 클라이언트 생성 실패: {e}")
    print("💡 환경변수를 확인하거나 'az login'을 실행하세요.")

🔑 API 키 인증을 사용합니다.
✅ Azure OpenAI 클라이언트 생성 성공!


## 4. 에이전트 생성

GroupChat에 참여할 세 개의 전문 에이전트를 생성합니다:
1. **Coordinator (Manager)**: 대화를 조율하고 다음 발언자를 선택
2. **Researcher**: 정보 수집 및 배경 조사 담당
3. **Writer**: 수집된 정보를 바탕으로 최종 답변 작성

In [14]:
print("🤖 에이전트 생성 중...\n")

# 1. Coordinator (Manager Agent)
# 주의: response_format은 GroupChatBuilder가 자동으로 ManagerSelectionResponse로 설정합니다
coordinator = ChatAgent(
    name="Coordinator",
    description="Coordinates multi-agent collaboration by selecting speakers",
    instructions="""
You coordinate a team conversation to solve the user's task.

Review the conversation history and select the next participant to speak.

Guidelines:
- Start with Researcher to gather information
- Then have Writer synthesize the final answer
- Only finish after both have contributed meaningfully
- Allow for multiple rounds of information gathering if needed
""",
    chat_client=_get_chat_client(),
)

print("✅ Coordinator (Manager) 에이전트 생성 완료")

# 2. Researcher Agent
researcher = ChatAgent(
    name="Researcher",
    description="Collects relevant background information",
    instructions="Gather concise facts that help a teammate answer the question.",
    chat_client=_get_chat_client(),
)

print("✅ Researcher 에이전트 생성 완료")

# 3. Writer Agent
writer = ChatAgent(
    name="Writer",
    description="Synthesizes polished answers from gathered information",
    instructions="Compose clear and structured answers using any notes provided.",
    chat_client=_get_chat_client(),
)

print("✅ Writer 에이전트 생성 완료")
print("\n🎉 모든 에이전트가 성공적으로 생성되었습니다!")

🤖 에이전트 생성 중...

🔑 API 키 인증을 사용합니다.
✅ Coordinator (Manager) 에이전트 생성 완료
🔑 API 키 인증을 사용합니다.
✅ Researcher 에이전트 생성 완료
🔑 API 키 인증을 사용합니다.
✅ Writer 에이전트 생성 완료

🎉 모든 에이전트가 성공적으로 생성되었습니다!


## 5. GroupChat 워크플로우 구축

`GroupChatBuilder`를 사용하여 Agent-Based Manager 워크플로우를 구성합니다:
- **set_manager()**: Coordinator 에이전트를 관리자로 설정
- **participants()**: 참여 에이전트들 등록
- **with_termination_condition()**: 종료 조건 설정

In [15]:
print("🔧 GroupChat 워크플로우 구축 중...\n")

# 종료 조건: Assistant 메시지가 2개 이상이면 종료
# (Researcher와 Writer가 각각 1번씩 응답하면 종료)
def termination_condition(messages: list[ChatMessage]) -> bool:
    """Assistant 역할의 메시지가 2개 이상이면 True 반환"""
    assistant_count = sum(1 for msg in messages if msg.role == Role.ASSISTANT)
    return assistant_count >= 2

# GroupChat 워크플로우 구축
workflow = (
    GroupChatBuilder()
    .set_manager(coordinator, display_name="Orchestrator")
    .participants([researcher, writer])
    .with_termination_condition(termination_condition)
    .build()
)

print("✅ GroupChat 워크플로우가 성공적으로 구축되었습니다!")
print("\n📋 워크플로우 구성:")
print("  - Manager: Coordinator (Orchestrator)")
print("  - Participants: Researcher, Writer")
print("  - Termination: Assistant 메시지 2개 이상")
print("\n💡 워크플로우가 실행 준비되었습니다!")

🔧 GroupChat 워크플로우 구축 중...

✅ GroupChat 워크플로우가 성공적으로 구축되었습니다!

📋 워크플로우 구성:
  - Manager: Coordinator (Orchestrator)
  - Participants: Researcher, Writer
  - Termination: Assistant 메시지 2개 이상

💡 워크플로우가 실행 준비되었습니다!


## 6. 워크플로우 실행 (기본 예제)

간단한 작업으로 GroupChat 워크플로우를 실행하고 결과를 확인합니다.

In [21]:
async def run_basic_example():
    """기본 GroupChat 실행 예제"""
    
    task = "파이썬에서 async/await를 사용하는 주요 이점은 무엇인가요? 간결하게 요약해 주세요."
    
    print("\n🚀 GroupChat with Agent-Based Manager 시작...\n")
    print(f"📝 작업: {task}\n")
    print("=" * 80)
    
    final_conversation: list[ChatMessage] = []
    last_executor_id: str | None = None
    
    try:
        # 워크플로우 스트리밍 실행
        async for event in workflow.run_stream(task):
            # 에이전트 실행 업데이트 이벤트 처리
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                
                # 새로운 에이전트가 시작되면 줄바꿈
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    print(f"\n[{eid}]: ", end="", flush=True)
                    last_executor_id = eid
                
                # 스트리밍 데이터 출력
                print(event.data, end="", flush=True)
            
            # 최종 결과 이벤트 처리
            elif isinstance(event, WorkflowOutputEvent):
                final_conversation = cast(list[ChatMessage], event.data)
        
        # 최종 대화 기록 출력
        if final_conversation and isinstance(final_conversation, list):
            print("\n\n" + "=" * 80)
            print("📋 최종 대화 기록")
            print("=" * 80)
            
            for i, msg in enumerate(final_conversation, 1):
                role = getattr(msg.role, "value", str(msg.role))
                author = msg.author_name or role
                text = (msg.text or "").strip()
                
                print(f"\n{i:02d}. [{author}] ({role})")
                print(f"    {text[:200]}{'...' if len(text) > 200 else ''}")
            
            print("\n" + "=" * 80)
            print("✅ GroupChat 실행 완료!")
        
        return final_conversation
    
    except Exception as e:
        print(f"\n❌ 실행 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

# 실행
result = await run_basic_example()

INFO:agent_framework._workflows._runner:Yielding pre-loop events
INFO:agent_framework._workflows._runner:Starting superstep 1
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'system'}, 'contents': [{'type': 'text', 'text': "\nYou coordinate a team conversation to solve the user's task.\n\nReview the conversation history and select the next participant to speak.\n\nGuidelines:\n- Start with Researcher to gather information\n- Then have Writer synthesize the final answer\n- Only finish after both have contributed meaningfully\n- Allow for multiple rounds of information gathering if needed\n"}], 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'system'}, 'contents': [{'type': 'text', 'text': 'Available participants:\n- Researcher: Collects relevant background information\n- Writer: Synthesizes polished answers from gathered information\n\nIMPORTANT: Choose only from these exact participant names (c


🚀 GroupChat with Agent-Based Manager 시작...

📝 작업: 파이썬에서 async/await를 사용하는 주요 이점은 무엇인가요? 간결하게 요약해 주세요.



INFO:httpx:HTTP Request: POST https://aistudioaiservices343274518006.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"



[groupchat_agent:Orchestrator]: {"selected_participant":"Researcher","instruction":"Collect and summarize the key benefits of using async/await in Python, and prepare to present them concisely for a Korean-speaking audience.","finish":false,"final_message":null}

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Researcher","instruction":"Collect and summarize the key benefits of using async/await in Python, and prepare to present them concisely for a Korean-speaking audience.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyZHB0kK6eE3e0RtfgdbJQUT7PEE', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Researcher","instruction":"Collect and summarize the key benefits of using async/await in Python, and prepare to present them concisely for a Korean-speaking audience.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyZHB0kK6eE3e0RtfgdbJQUT7PEE', 'additional_properties': {}}
INFO:agent_framework._workflows._r



[groupchat_agent:Researcher]: 파이썬에서 async/await를 사용하는 주요 이점:

- **동시성 향상:** 많은 I/O 작업을 효율적으로 처리할 수 있습니다.
- **코드 가독성:** 콜백이나 복잡한 구조 없이 직관적이고 깔끔한 코드를 작성할 수 있습니다.
- **자원 효율성:** 스레드 사용 없이 메모리와 CPU 소모를 줄일 수 있습니다.
- **비동기 처리:** 작업 대기 중에도 다른 작업을 계속할 수 있습니다.

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '파이썬에서 async/await를 사용하는 주요 이점:\n\n- **동시성 향상:** 많은 I/O 작업을 효율적으로 처리할 수 있습니다.\n- **코드 가독성:** 콜백이나 복잡한 구조 없이 직관적이고 깔끔한 코드를 작성할 수 있습니다.\n- **자원 효율성:** 스레드 사용 없이 메모리와 CPU 소모를 줄일 수 있습니다.\n- **비동기 처리:** 작업 대기 중에도 다른 작업을 계속할 수 있습니다.'}], 'author_name': 'Researcher', 'message_id': 'chatcmpl-CmyZLXTnpZMuH3B59e1MSWZamfXnw', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '파이썬에서 async/await를 사용하는 주요 이점:\n\n- **동시성 향상:** 많은 I/O 작업을 효율적으로 처리할 수 있습니다.\n- **코드 가독성:** 콜백이나 복잡한 구조 없이 직관적이고 깔끔한 코드를 작성할 수 있습니다.\n- **자원 효율성:** 스레드 사용 없이 메모리와 CPU 소모를 줄일 수 있습니다.\n- **비동기 처리:** 작업 대기 중에도 다른 작업을 계속할 수 있습니다.'}], 'author_name': 'Researcher', 'message_id': 'chatcmpl-CmyZLXTnpZMuH3B59e1MSWZamfXnw', 'additional_properties': {}}
INFO:agent_framework._workflows._runner:Comple



[groupchat_agent:Orchestrator]: {"selected_participant":null,"instruction":null,"finish":true,"final_message":"파이썬에서 async/await를 사용하는 주요 이점은 다음과 같습니다:\n\n- 많은 I/O 작업을 효율적으로 동시에 처리할 수 있어 프로그램의 성능과 반응성이 향상됩니다.\n- 코드가 간결하고 이해하기 쉬워져 유지보수가 용이합니다.\n- 스레드나 프로세스보다 적은 자원을 사용해 시스템 부담이 줄어듭니다.\n- try/except로 에러 관리를 쉽게 할 수 있어 안정적인 비동기 프로그래밍이 가능합니다.\n\n즉, async/await는 효율적이고 직관적인 비동기 프로그래밍을 돕습니다."}

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":null,"instruction":null,"finish":true,"final_message":"파이썬에서 async/await를 사용하는 주요 이점은 다음과 같습니다:\\n\\n- 많은 I/O 작업을 효율적으로 동시에 처리할 수 있어 프로그램의 성능과 반응성이 향상됩니다.\\n- 코드가 간결하고 이해하기 쉬워져 유지보수가 용이합니다.\\n- 스레드나 프로세스보다 적은 자원을 사용해 시스템 부담이 줄어듭니다.\\n- try/except로 에러 관리를 쉽게 할 수 있어 안정적인 비동기 프로그래밍이 가능합니다.\\n\\n즉, async/await는 효율적이고 직관적인 비동기 프로그래밍을 돕습니다."}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyZNxMrJDDgkau6qiPNraG4ixhTK', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":null,"instruction":null,"finish":true,"final_message":"파이썬에서 async/await를 사용하는 주요 이점은 다음과 같습니다:\\n\\n- 많은 I/O 작업을 효율적으로 동시에 처리할 수 있어 프로그램의 성능과 반응성이 향상됩니다.\\n- 코드가 간결하고 이해하기 쉬워져 유지보수가 용이합니다.\\n- 스레드나 프로세스보다 적은 자원을 사용해 시스템 부담이 줄어듭



📋 최종 대화 기록

01. [user] (user)
    파이썬에서 async/await를 사용하는 주요 이점은 무엇인가요? 간결하게 요약해 주세요.

02. [Orchestrator] (assistant)
    Collect and summarize the key benefits of using async/await in Python, and prepare to present them concisely for a Korean-speaking audience.

03. [Researcher] (assistant)
    파이썬에서 async/await를 사용하는 주요 이점:

- **동시성 향상:** 많은 I/O 작업을 효율적으로 처리할 수 있습니다.
- **코드 가독성:** 콜백이나 복잡한 구조 없이 직관적이고 깔끔한 코드를 작성할 수 있습니다.
- **자원 효율성:** 스레드 사용 없이 메모리와 CPU 소모를 줄일 수 있습니다.
- **비동기 처리:** 작업 대기 중에...

04. [Orchestrator] (assistant)
    파이썬에서 async/await를 사용하는 주요 이점은 다음과 같습니다:

- 많은 I/O 작업을 효율적으로 동시에 처리할 수 있어 프로그램의 성능과 반응성이 향상됩니다.
- 코드가 간결하고 이해하기 쉬워져 유지보수가 용이합니다.
- 스레드나 프로세스보다 적은 자원을 사용해 시스템 부담이 줄어듭니다.
- try/except로 에러 관리를 쉽게 할 수 있어 ...

✅ GroupChat 실행 완료!


## 7. 커스텀 작업 실행

다양한 작업으로 GroupChat의 유연성을 테스트해봅시다.

In [ ]:
async def run_custom_task(task: str, max_assistant_messages: int = 2):
    """
    커스텀 작업으로 GroupChat 실행
    
    Args:
        task: 실행할 작업 내용
        max_assistant_messages: 종료 조건 (Assistant 메시지 개수)
    """
    
    print(f"\n🚀 커스텀 작업 실행\n")
    print(f"📝 작업: {task}")
    print(f"🎯 종료 조건: Assistant 메시지 {max_assistant_messages}개\n")
    print("=" * 80)
    
    # 종료 조건 동적 생성
    def custom_termination(messages: list[ChatMessage]) -> bool:
        count = sum(1 for msg in messages if msg.role == Role.ASSISTANT)
        return count >= max_assistant_messages
    
    # 새로운 워크플로우 구축
    custom_workflow = (
        GroupChatBuilder()
        .set_manager(coordinator, display_name="Orchestrator")
        .with_termination_condition(custom_termination)
        .participants([researcher, writer])
        .build()
    )
    
    final_conversation: list[ChatMessage] = []
    last_executor_id: str | None = None
    
    try:
        async for event in custom_workflow.run_stream(task):
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    print(f"\n[{eid}]: ", end="", flush=True)
                    last_executor_id = eid
                print(event.data, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                final_conversation = cast(list[ChatMessage], event.data)
        
        print("\n\n" + "=" * 80)
        print(f"✅ 작업 완료! (총 메시지: {len(final_conversation)})")
        print("=" * 80)
        
        return final_conversation
    
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        return None

# 예제 실행
print("\n📚 예제 1: Python 데코레이터 설명")
result1 = await run_custom_task(
    "실제 예제를 통해 Python 데코레이터를 설명하세요.",
    max_assistant_messages=2
)

INFO:agent_framework._workflows._runner:Yielding pre-loop events
INFO:agent_framework._workflows._runner:Starting superstep 1
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'system'}, 'contents': [{'type': 'text', 'text': "\nYou coordinate a team conversation to solve the user's task.\n\nReview the conversation history and select the next participant to speak.\n\nGuidelines:\n- Start with Researcher to gather information\n- Then have Writer synthesize the final answer\n- Only finish after both have contributed meaningfully\n- Allow for multiple rounds of information gathering if needed\n"}], 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'system'}, 'contents': [{'type': 'text', 'text': 'Available participants:\n- Researcher: Collects relevant background information\n- Writer: Synthesizes polished answers from gathered information\n\nIMPORTANT: Choose only from these exact participant names (c


📚 예제 1: Python 데코레이터 설명

🚀 커스텀 작업 실행

📝 작업: Explain Python decorators with a practical example.
🎯 종료 조건: Assistant 메시지 2개



INFO:httpx:HTTP Request: POST https://aistudioaiservices343274518006.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"



[groupchat_agent:Orchestrator]: {"selected_participant":"Researcher","instruction":"Please gather information on Python decorators and provide a practical, beginner-friendly example.","finish":false,"final_message":null}

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Researcher","instruction":"Please gather information on Python decorators and provide a practical, beginner-friendly example.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXZWqNTqeO14kbFS4pG5gC01jDL', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Researcher","instruction":"Please gather information on Python decorators and provide a practical, beginner-friendly example.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXZWqNTqeO14kbFS4pG5gC01jDL', 'additional_properties': {}}
INFO:agent_framework._workflows._runner:Completed superstep 1
INFO:agent_framework._workflows._runner:Starting superst



[groupchat_agent:Researcher]: **Facts about Python decorators:**

- A decorator in Python is a function that takes another function as input and returns a new function with added functionality, without modifying the original function’s code.
- Decorators use the `@decorator_name` syntax placed above a function definition.
- They are commonly used for logging, access control, timing functions, or modifying input/output.

**Practical example: Logging function calls**

```python
def my_decorator(func):
    def wrapper():
        print("Function is about to be called.")
        func()
        print("Function call finished.")
    return wrapper

@my_decorator
def greet():
    print("Hello!")

greet()
```

**What happens:**
- When you call `greet()`, the decorator adds logging before and after the original function.
- Output:
  ```
  Function is about to be called.
  Hello!
  Function call finished.
  ```

**Summary:**  
Decorators make it easy to add reusable behavior around functions wit

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '**Facts about Python decorators:**\n\n- A decorator in Python is a function that takes another function as input and returns a new function with added functionality, without modifying the original function’s code.\n- Decorators use the `@decorator_name` syntax placed above a function definition.\n- They are commonly used for logging, access control, timing functions, or modifying input/output.\n\n**Practical example: Logging function calls**\n\n```python\ndef my_decorator(func):\n    def wrapper():\n        print("Function is about to be called.")\n        func()\n        print("Function call finished.")\n    return wrapper\n\n@my_decorator\ndef greet():\n    print("Hello!")\n\ngreet()\n```\n\n**What happens:**\n- When you call `greet()`, the decorator adds logging before and after the original function.\n- Output:\n  ```\n  Function is about to be called



[groupchat_agent:Orchestrator]: {"selected_participant":"Writer","instruction":"Please synthesize a clear explanation about Python decorators based on the collected information, including the practical logging example and explanation.","finish":false,"final_message":null}

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Writer","instruction":"Please synthesize a clear explanation about Python decorators based on the collected information, including the practical logging example and explanation.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXeSOsk1aE8sFzPlTKPcVlQBSmR', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Writer","instruction":"Please synthesize a clear explanation about Python decorators based on the collected information, including the practical logging example and explanation.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXeSOsk1aE8sFzPlTKPcVlQBSmR', 'additional_properties': {}}
INFO:agent_fra



✅ 작업 완료! (총 메시지: 4)


In [18]:
# 추가 예제
print("\n🌐 예제 2: 웹 개발 프레임워크 비교")
result2 = await run_custom_task(
    "Compare FastAPI and Flask for building REST APIs. Which one is better for beginners?",
    max_assistant_messages=3
)

INFO:agent_framework._workflows._runner:Yielding pre-loop events
INFO:agent_framework._workflows._runner:Starting superstep 1
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'system'}, 'contents': [{'type': 'text', 'text': "\nYou coordinate a team conversation to solve the user's task.\n\nReview the conversation history and select the next participant to speak.\n\nGuidelines:\n- Start with Researcher to gather information\n- Then have Writer synthesize the final answer\n- Only finish after both have contributed meaningfully\n- Allow for multiple rounds of information gathering if needed\n"}], 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'system'}, 'contents': [{'type': 'text', 'text': 'Available participants:\n- Researcher: Collects relevant background information\n- Writer: Synthesizes polished answers from gathered information\n\nIMPORTANT: Choose only from these exact participant names (c


🌐 예제 2: 웹 개발 프레임워크 비교

🚀 커스텀 작업 실행

📝 작업: Compare FastAPI and Flask for building REST APIs. Which one is better for beginners?
🎯 종료 조건: Assistant 메시지 3개



INFO:httpx:HTTP Request: POST https://aistudioaiservices343274518006.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"



[groupchat_agent:Orchestrator]: {"selected_participant":"Researcher","instruction":"Gather information comparing FastAPI and Flask for building REST APIs, focusing on features, ease of use, performance, and which is more suitable for beginners.","finish":false,"final_message":null}

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Researcher","instruction":"Gather information comparing FastAPI and Flask for building REST APIs, focusing on features, ease of use, performance, and which is more suitable for beginners.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXgaKXebNjx3vftAhgAMNDueZM9', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Researcher","instruction":"Gather information comparing FastAPI and Flask for building REST APIs, focusing on features, ease of use, performance, and which is more suitable for beginners.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXgaKXebNjx3vftAhgAMNDueZM9', 'additional_properties'

...



[groupchat_agent:Orchestrator]: {"selected_participant":"Writer","instruction":"Synthesize a clear comparison of FastAPI and Flask based on the research, with a recommendation for beginners.","finish":false,"final_message":null}

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Writer","instruction":"Synthesize a clear comparison of FastAPI and Flask based on the research, with a recommendation for beginners.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXpS4PuxtimUqgPOMvEkkffZ11o', 'additional_properties': {}}
INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":"Writer","instruction":"Synthesize a clear comparison of FastAPI and Flask based on the research, with a recommendation for beginners.","finish":false,"final_message":null}'}], 'author_name': 'Coordinator', 'message_id': 'chatcmpl-CmyXpS4PuxtimUqgPOMvEkkffZ11o', 'additional_properties': {}}
INFO:agent_framework._workflows._runner:Completed superstep 5
INFO:agent_framework._workflows._runner:

{"selected_participant":null,"instruction":null,"finish":true,"final_message":"Here’s a comparison of FastAPI and Flask for building REST APIs, with a recommendation for beginners:\n\n**Flask** is a well-established, simple Python framework. It’s easy to learn, has a huge community, and plenty of resources, making it ideal for beginners who want to start building APIs quickly without additional concepts.\n\n**FastAPI** is modern and powerful. It offers automatic data validation, type hints, async support, and interactive documentation out of the box. However, it expects the developer to be familiar with Python typing and some async concepts, which can add a layer of complexity for absolute beginners, despite its helpful features.\n\n**Conclusion:** If you’re new to web development and building REST APIs, Flask is generally better for beginners due to its simplicity and extensive support. If you want cutting-edge features and don’t mind learning a bit more up front, FastAPI is an excell

INFO:agent_framework:{'type': 'chat_message', 'role': {'type': 'role', 'value': 'assistant'}, 'contents': [{'type': 'text', 'text': '{"selected_participant":null,"instruction":null,"finish":true,"final_message":"Here’s a comparison of FastAPI and Flask for building REST APIs, with a recommendation for beginners:\\n\\n**Flask** is a well-established, simple Python framework. It’s easy to learn, has a huge community, and plenty of resources, making it ideal for beginners who want to start building APIs quickly without additional concepts.\\n\\n**FastAPI** is modern and powerful. It offers automatic data validation, type hints, async support, and interactive documentation out of the box. However, it expects the developer to be familiar with Python typing and some async concepts, which can add a layer of complexity for absolute beginners, despite its helpful features.\\n\\n**Conclusion:** If you’re new to web development and building REST APIs, Flask is generally better for beginners due t



✅ 작업 완료! (총 메시지: 5)


## 8. 대화 기록 분석

GroupChat 실행 결과를 분석하는 유틸리티 함수들을 정의합니다.

In [19]:
def analyze_conversation(conversation: list[ChatMessage]) -> dict:
    """
    대화 기록을 분석하여 통계 정보를 반환합니다.
    
    Args:
        conversation: 분석할 대화 메시지 리스트
    
    Returns:
        분석 결과 딕셔너리
    """
    if not conversation:
        return {}
    
    # 역할별 메시지 카운트
    role_counts = {}
    # 발언자별 메시지 카운트
    author_counts = {}
    # 총 단어 수
    total_words = 0
    
    for msg in conversation:
        # 역할 카운트
        role = getattr(msg.role, "value", str(msg.role))
        role_counts[role] = role_counts.get(role, 0) + 1
        
        # 발언자 카운트
        author = msg.author_name or role
        author_counts[author] = author_counts.get(author, 0) + 1
        
        # 단어 수 계산
        if msg.text:
            total_words += len(msg.text.split())
    
    return {
        "total_messages": len(conversation),
        "role_counts": role_counts,
        "author_counts": author_counts,
        "total_words": total_words,
        "avg_words_per_message": total_words / len(conversation) if conversation else 0
    }

def print_conversation_analysis(conversation: list[ChatMessage]):
    """대화 분석 결과를 출력합니다."""
    analysis = analyze_conversation(conversation)
    
    if not analysis:
        print("분석할 대화가 없습니다.")
        return
    
    print("\n📊 대화 분석 결과")
    print("=" * 60)
    print(f"총 메시지 수: {analysis['total_messages']}")
    print(f"총 단어 수: {analysis['total_words']}")
    print(f"메시지당 평균 단어 수: {analysis['avg_words_per_message']:.1f}")
    
    print("\n📋 역할별 메시지 분포:")
    for role, count in analysis['role_counts'].items():
        print(f"  - {role}: {count}개")
    
    print("\n👥 발언자별 메시지 분포:")
    for author, count in analysis['author_counts'].items():
        print(f"  - {author}: {count}개")
    
    print("=" * 60)

# 이전 결과 분석
if result:
    print_conversation_analysis(result)


📊 대화 분석 결과
총 메시지 수: 4
총 단어 수: 192
메시지당 평균 단어 수: 48.0

📋 역할별 메시지 분포:
  - user: 1개
  - assistant: 3개

👥 발언자별 메시지 분포:
  - user: 1개
  - Orchestrator: 2개
  - Researcher: 1개


## 9. GroupChat의 주요 개념 정리

이번 노트북에서 학습한 핵심 개념들을 정리합니다.

In [ ]:
print("📚 GroupChat 주요 개념 정리")
print("=" * 80)

concepts = {
    "1. Agent-Based Manager": [
        "- 일반 함수 대신 완전한 ChatAgent를 관리자로 사용",
        "- 도구, 컨텍스트, 관찰 가능성 등 모든 에이전트 인프라 활용 가능",
        "- 구조화된 출력(ManagerSelectionResponse)으로 의사결정 명확화",
        "- set_manager() 메서드로 설정"
    ],
    "2. ManagerSelectionResponse": [
        "- 다음 발언자를 선택하기 위한 구조화된 응답 형식",
        "- selected_participant: 다음에 말할 에이전트 이름",
        "- instruction: 선택된 에이전트에게 전달할 지시사항",
        "- finish: 대화 종료 여부",
        "- final_message: 종료 시 최종 메시지"
    ],
    "3. 종료 조건 (Termination Condition)": [
        "- with_termination_condition()으로 설정",
        "- 대화 메시지 리스트를 받아 bool 반환하는 함수",
        "- 조건 충족 시 관리자가 최종 메시지 제공 기회 받음",
        "- 예: 특정 횟수 이상 응답, 특정 키워드 포함 등"
    ],
    "4. 참여자 (Participants)": [
        "- participants() 메서드로 등록",
        "- 리스트 또는 딕셔너리 형태로 전달 가능",
        "- 각 참여자는 고유한 이름을 가져야 함",
        "- ChatAgent 또는 Executor 인스턴스 가능"
    ],
    "5. 스트리밍 이벤트": [
        "- AgentRunUpdateEvent: 에이전트 실행 중 실시간 업데이트",
        "- WorkflowOutputEvent: 워크플로우 최종 출력",
        "- WorkflowInputEvent: 워크플로우 입력 (드물게 발생)",
        "- WorkflowErrorEvent: 오류 발생 시"
    ]
}

for concept, points in concepts.items():
    print(f"\n{concept}")
    print("-" * 70)
    for point in points:
        print(point)

print("\n" + "=" * 80)

# 모범 사례
print("\n✨ 모범 사례")
print("=" * 80)
best_practices = [
    "1. 관리자 지시사항을 명확하게 작성하여 적절한 조율 유도",
    "2. 각 에이전트의 역할과 전문성을 명확히 정의",
    "3. 적절한 종료 조건을 설정하여 무한 루프 방지",
    "4. 스트리밍 이벤트를 활용하여 실시간 진행 상황 표시",
    "5. 대화 기록을 분석하여 워크플로우 개선",
    "6. Azure CLI 인증 또는 API 키 인증 중 환경에 맞는 방식 선택",
    "7. 에러 처리를 통해 안정적인 실행 보장"
]

for practice in best_practices:
    print(f"  ✅ {practice}")

print("\n" + "=" * 80)

## 11. 실습 과제

다음 실습 과제를 통해 GroupChat 개념을 더 깊이 이해해보세요.

In [ ]:
print("🎯 실습 과제")
print("=" * 80)

exercises = {
    "초급": [
        "1. 새로운 전문 에이전트(예: Critic, Tester)를 추가하고 GroupChat에 통합",
        "2. 다른 종료 조건을 실험 (예: 특정 단어 포함, 메시지 길이 등)",
        "3. 다양한 작업으로 워크플로우 테스트 및 결과 비교"
    ],
    "중급": [
        "1. 관리자의 지시사항을 수정하여 다른 조율 전략 시도",
        "2. 대화 기록 분석 기능을 확장하여 더 상세한 통계 제공",
        "3. 에이전트별 도구(tools)를 추가하여 기능 확장"
    ],
    "고급": [
        "1. 동적으로 에이전트를 추가/제거하는 시스템 구현",
        "2. 외부 API와 연동하는 커스텀 에이전트 개발",
        "3. 여러 GroupChat을 연결하는 메타 워크플로우 구축"
    ]
}

for level, tasks in exercises.items():
    print(f"\n{level} 실습:")
    for task in tasks:
        print(f"  {task}")

print("\n" + "=" * 80)

# 실습 템플릿
print("\n📝 실습 템플릿")
print("=" * 80)
print("""
# 실습을 위한 빈 코드 블록

# TODO: 여기에 실습 코드를 작성하세요

# 예제: 새로운 에이전트 추가
# critic = ChatAgent(
#     name="Critic",
#     description="Reviews and provides constructive feedback",
#     instructions="Review the work and provide constructive feedback.",
#     chat_client=_get_chat_client(),
# )

# 새로운 워크플로우 구축
# custom_workflow = (
#     GroupChatBuilder()
#     .set_manager(coordinator, display_name="Orchestrator")
#     .participants([researcher, writer, critic])  # critic 추가
#     .with_termination_condition(your_condition_function)
#     .build()
# )

# 실행
# result = await run_custom_task("Your task here", custom_workflow)
""")

print("\n" + "=" * 80)

## 12. 요약 및 다음 단계

🎉 **축하합니다!** GroupChat with Agent-Based Manager의 핵심 개념을 모두 학습했습니다.

### 학습한 내용:
1. ✅ **GroupChatBuilder**: 다중 에이전트 워크플로우 구축
2. ✅ **Agent-Based Manager**: 에이전트를 관리자로 사용하는 고급 패턴
3. ✅ **ManagerSelectionResponse**: 구조화된 의사결정
4. ✅ **종료 조건**: 워크플로우 완료 시점 제어
5. ✅ **스트리밍 이벤트**: 실시간 진행 상황 모니터링
6. ✅ **대화 분석**: 결과 평가 및 개선

### 다음 단계:
- 📚 [Agent Framework 공식 문서](https://github.com/microsoft/agent-framework) 참고
- 🔧 더 복잡한 에이전트 팀 구성 실험
- 🌐 실제 비즈니스 문제에 적용
- 📊 성능 모니터링 및 최적화
- 🧪 다양한 관리 전략 테스트

### 관련 노트북:
- **11. magentic.ipynb**: Magentic 워크플로우 (GroupChat의 확장)
- **09. sequential.ipynb**: 순차적 워크플로우
- **10. concurrent.ipynb**: 병렬 워크플로우

**즐거운 에이전트 개발 되세요! 🚀**